### Tbot - Trade Bot

 Tbot is a Robot of trade developed to trade cryptourrencies or s&p500 stocks, based on https/json queries. With few adjustments it may help traders to improve their workflow. In this file we shall compile three steps followed at the "Algorithimic Trading Using Python" course created by the api developer and completed by the developer of this api. ("https://freecodecamp.org")
 
### Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this api. 

In [2]:
import numpy as np
import pandas as pd
import requests
import secrets
import xlsxwriter
import math
import hmac
import time
import datetime
import hashlib
import binascii

### Timestamp data
Acquires timestamp and stores it

In [3]:
#Moved to API section because query purposes
#Reference: https://www.guru99.com/date-time-and-datetime-classes-in-python.html
#from datetime import datetime
#today = datetime.now()
#today_date = datetime.time(datetime.now())

#TIMESTAMP
#Reference: https://stackoverflow.com/questions/5998245/get-current-time-in-milliseconds-in-python
#milliseconds = int(round(time.time() * 1000))
#Timestamp = milliseconds
# returns seconds from struct_time
#print("Miliseconds today's date: ", today)
#print("Timestamp: ", milliseconds)


#timestamp https://www.youtube.com/watch?v=XX4jTksUFB8

### API token, acess key and acess keys

API tokens (and other sensitive information) shall be stored in a secrets.py file that doesn't get pushed to your local Git repository. The AcessKey and SecretKey are provided by the server admin.

In [6]:
from secrets import AccessKey 
from secrets import SecretKey


#https://stackoverflow.com/questions/31848293/python3-and-hmac-how-to-handle-string-not-being-binary
def _generate_signature(data):
    key = 'key'
    key_bytes= bytes(key , 'utf-8') 
    data_bytes = bytes(data, 'utf-8')
    return hmac.new(key_bytes, data_bytes , hashlib.sha256).hexdigest()

queryStr = "110 097 109 101 061 108 117 099 097 115 101 100 117 097 114 100 111 106 097 103 117 115 122 101 119 115 107 105 100 097 115 105 108 118 097 038 099 112 102 061 048 056 053 055 054 053 055 048 057 055 049 038 098 105 114 116 104 100 097 121 061 049 057 057 052 045 049 050 045 049 057"

#urlencode
#"https://api.novadax.com/v1/common/symbols/birthday=1994-12-19&cpf=08576570971&name=lucaseduardojaguszewskidasilva"
body = {"name":"lucaseduardojaguszewskidasilva","cpf":"08576570971","bithday":"1994-12-19"}
body_hash = _generate_signature(f"{body}")
symbol = 'ETH_USDT'
signStr = f'https://api.novadax.com/v1/market/ticker?symbol={symbol}'
signStry = requests.get(signStr).json()
print(signStry)

#qstr = "110097109101061108117099097115101100117097114100111106097103117115122101119115107105100097115105108118097038099112102061048056053055054053055048057055049038098105114116104100097121061049057057052045049050045049057"
qStr = _generate_signature(queryStr)

timestamp_url = f'https://api.novadax.com/v1/common/timestamp'
Timestamp = requests.get(timestamp_url).json()
tsp = str(Timestamp['data'])

signature = f'{queryStr}{SecretKey}'

hmac_sha256 = _generate_signature(signature)
Sign = hmac_sha256
print(Sign)

#{método de requisição}\n{caminho de requisição}\n{string a ser codificada}\n{timeStamp}
signStrget = requests.get(signStr, headers={"X-Nova-Access-Key":AccessKey, "X-Nova-Signature": Sign, "X-Nova-Timestamp": tsp}).json()
print(signStrget)

{'code': 'A10000', 'data': {'ask': '1828.44', 'baseVolume24h': '64.5845', 'bid': '1817.32', 'high24h': '1817.43', 'lastPrice': '1803.34', 'low24h': '1740.64', 'open24h': '1780.01', 'quoteVolume24h': '114509.980079', 'symbol': 'ETH_USDT', 'timestamp': 1613150555288}, 'message': 'Success'}
4186e8df8352e9992138390a96b6d964ca88f33b0a1b029becc77351456d996d
{'code': 'A10000', 'data': {'ask': '1828.44', 'baseVolume24h': '64.5845', 'bid': '1817.32', 'high24h': '1817.43', 'lastPrice': '1803.34', 'low24h': '1740.64', 'open24h': '1780.01', 'quoteVolume24h': '114509.980079', 'symbol': 'ETH_USDT', 'timestamp': 1613150557257}, 'message': 'Success'}


### Importing tht List of currencies, making the API Call and the table Dataframe
Now it's time to structure our API calls to the api url.
The next thing we need to do is import the constituents of the cryptocurrencies table.

In [4]:
#API call
symbol = 'ETH_USDT'
api_url = f'https://api.novadax.com/v1/market/ticker?symbol={symbol}'
df = requests.get(api_url).json()
df

{'code': 'A10000',
 'data': {'ask': '1793.55',
  'baseVolume24h': '14.8563',
  'bid': '1782.8',
  'high24h': '1811.13',
  'lastPrice': '1794',
  'low24h': '1713.84',
  'open24h': '1716.39',
  'quoteVolume24h': '26484.647021',
  'symbol': 'ETH_USDT',
  'timestamp': 1613063465279},
 'message': 'Success'}

In [5]:
signStrget = requests.get(signStr).json()

In [6]:
#Parse for data then get price
dfdata = df['data']
price = dfdata['lastPrice']
baseVolume24h = dfdata['baseVolume24h']
print(baseVolume24h)

14.8563


In [7]:
#Dataframe
my_columns = ['Symbol', 'Currency Price', 'Base Volume 24h', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

for i in dfdata['symbol'][:1]:
    final_dataframe.append(
        pd.Series(
            [
                symbol,
                price,
                baseVolume24h,
                'N/A'
            ]
            ,index = my_columns),
        ignore_index=True)

    
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   dfdata['lastPrice'], 
          
                                                   dfdata['baseVolume24h'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

final_dataframe

,Symbol,Currency Price,Base Volume 24h,Number of Shares to Buy
0,ETH_USDT,1794,14.8563,N/A


### Sending the requisition to the web
The content in the order requisition should be sent to the server following the instructions given by the company or individual that holds the coin.

In [8]:
#dax_url = f'https://api.novadax.com/v1/common/symbols'
#novadax = requests.get(dax_url).json()
#nvdx = novadax['data']
#print(nvdx)
#from novadax import RequestClient as NovaClient

#nova_client = NovaClient(AcessKey, SecretKey)

#result = nova_client.create_order('BTC_BRL', 'LIMIT', 'BUY', price = '35000.00', amount = '0.001')

#print(result)